## 예문 생성

-  sent.db에서 신어에 해당하는 예문을 가져온뒤 Bidirectional GRU Model을 사용하여 예문 추출

In [ ]:
import sqlite3
import pandas as pd
from collections import defaultdict
import Example
from sklearn.model_selection import train_test_split

In [ ]:
conn = sqlite3.connect('sent.db') # sent.db 연결
# sent.db에는 신어, 신어가 포함된 예문이 들어 있음
temp = pd.read_sql('SELECT * FROM sent', conn)  # sent.db에서 전체 데이터를 temp에 저장

full_sentence = list()
example_sentence = list()
for _ in range(len(temp['0'])):
    sent = temp['0'][_] # _번째 문장들
    sent = sent.split('  ') # _번째 문장들은 더블스페이스 단위로 분리
    sent = [_+'/' for _ in sent] # 분리된 문장을 '/' 로 구분
    words = {temp['index'][_]: 1.0} # tme['index']의 _번째 행을 1로저장하는 dict 생성
    
    lgen = Example.generator_ltokenizer(words, sent) # Example 모듈에서 ltokenizer사용
                                                     # cohesion_forward 값을 기준으롬 명사 추출
    sentences = [[word for word in lgen.tokenizer(sentence)] for sentence in sent]
    # 각 신어의 예문을 토큰화 시켜서 sentences에 저장
    lgen.train_word_model(sentences) # 모델학습
    X,y = lgen.create_var(sentences) # X, y 값 지정
    lgen.model()
    lgen.model.fit(X, y, epochs=70, verbose=2)
    
    temp_sent = lgen.sentence_generation(temp['index'][_], 22) # 각단어에 해당하는 22 단어로 이루어진 문장 생성
    result = ''
    for _ in temp_sent: # temp_sent에서 /로 끝나면 break
        if _ == '/':
            break
        else:
            result = result + '' + _
    full_sentence.append(temp_sent)
    example_sentence.append(result)